# Ch6. Beyond Born-Oppenheimer

## Install

In [ ]:
#Installment

## 6.1. Non-Born-Oppenheimer molecular Hamiltonian

Section 4에서 본 Born-Oppenheimer approximation을 다시보자. laboratory frame coordinates에서의 Hamiltonian $\hat{H}_{LAB}$은 다음과 같이 표현할 수 있다.
 $$ \hat{H} = -\sum_I \frac{\nabla_{R_I}^{2}}{M_I} -\sum_i \frac{\nabla_{r_i}^{2}}{m_e} -\sum_{I} \sum_{i} \frac{Z_I e^2}{|R_I - r_i|} + \sum_i \sum_{j > i} \frac{e^2}{|r_i - r_j|} + \sum_I \sum_{J > I} \frac{Z_I Z_J e^2}{|R_I - R_J|} $$

where
- $\nabla_{r_i}^{2}$ and $\nabla_{R_I}^{2}$ are the second derivate operator with respecto to the position coordinates for electrons and nuclei, that is, $\nabla_{R_I}^{2} = \frac{\partial^2}{\partial_{x_I}^{2}} + \frac{\partial^2}{\partial_{y_I}^{2}} + \frac{\partial^2}{\partial_{z_I}^{2}}$ and likewise for the p<sup>th</sup> electrons.
- $r_{ij} = \left|r_i - r_j\right|, r_{iI} = \left|r_i - R_I\right|, R_{IJ} = \left|R_I - R_J\right|$ are the distances between electrons $i$ and $j$, electron $i$ and nucles $I$, and the nuclei $I$ and $J$ determined by the Eulidean norm
- indexes with upper case such as $I, J$ represent nuclei, and indexes with lower case such as $i, j$ represent electrons
- $Z_I$ represents the atomic number of the nuclei

위의 Hamiltonian에서 center-of-mass (COM) motion은 초기상태의 에너지를 바꾸지 않는다. 초기 상태는 양자화되어 있고 translation에 대하여 변하지 않는다. 그리고 이런 상태는 translation과 rotational motion에 영향을 받지 않는다. 이 과정에서 핵은 아직 COM 근처에서 진동하거나 제자리 회전을 통해 움직일 수 있다.

BO approximation은 핵의 움직임이 전자의 움직임과 결합되지 않는다고 가정한다. 이 말은 핵의 움직임에 대한 식 (회전과 진동) 그리고 전자의 움직임에 대한 식이 다음과 같다는 것을 의미한다.
$$
|\Psi_{total}(r,s,R)\rangle = \Psi_{rotational}(R)\Psi_{vibrational}(R)\Psi_{elec}(r,s:R) 
$$
여기에서 $R = {R_A,R_B,...,R_M}$는 핵의 좌표이고, $r = {r_p,r_i,r_j,...r_N}$은 전자의 좌표이다. 그리고 $s = {s_p,s_i,s_j,...,s_N}$는 spin 좌표이고, 전자의 wave function ($\Psi_{elec}(r,s;R)$)은 핵의 좌표(R)에 영향을 받는다.

non-BO method에선 전체 wave functin은 아직 rotation과 vibration 그리고 electronic energy level의 term으로 분리할 수 있지만 모든 에너지 level이 전자와 핵의 변수, spatial과 spin의 변수에 영향을 받는다.
$$
|\Psi_{total}(r,s,R)\rangle = \Psi_{rotational}(r,s,R,S)\Psi_{vibrational}(r,s,R,S)\Psi_{elec}(r,s,R,S) 
$$

그리고 BO approximation에서 핵의 자체 회전과 진동을 고려하기 위해서 핵의 위치를 고정하여 전자의 식만을 반복하여 풀 수 있다. 각 반복에서 원자의 핵은 고정되어 있고 움직이지 않기 때문에 Heisenberg의 uncertainty proinciple을 위반한다고 생각할 수 있다. 입자가 어디 있는지를 조금 더 정확하게 알면 그 입자의 momentum은 더 부정확 값을 얻는다. 일반적으로 내부 좌표 system은 분자의 가장 무거운 원자나 COM에 위치할 수 있다. 이 근사는 cover해야할 제한이 있고 특별히 진동의 스펙트럼 에너지에서 그렇다.
이번 챕터에서 non-BO method는 explicitly correlated Gaussian basis function(ECGs)를 활용하여 모든 입자 
explicit correlation의 예측된 power와 effectiveness를 사용하여 원자와 분자의 현상을 묘사한다. 그리고 이 방법은 적은 수의 입자 모델에서 사용될 수 있다. 즉, 높은 정확도 원자의 이온화 에너지(Rydberg states)나 excited state에서 회전과 진동을 높은 정확도로 알아내는 것과 같은 3개의 핵이나 7개보다 많은 전자에서 사용될 수 있다. 게다가 이 방법은 effective parallelization을 잘 처리할 수 있다.

### Internal Hamiltonian operator

N개의 입자가 있는(핵 하나와 N-1의 전자를 가진) 일반적인 non-relativistic 원자 시스템을 생각해보자. 

Converting from the laboratory frame Hamiltonian 
to the internal Hamiltonian involves rigorously separating the COM motion from the 
laboratory frame. We define an internal Cartesian coordinate frame (CCF) where all 
particles are treated on equal footing and all particles are allowed to roam space freely 
without any constraints. The change of coordinates from laboratory CCF to internal CCF 
is illustrated in Figure 6.2

laboratory fram에 Hamiltonian에서 internal Hamiltonian으로 변환은 laboratory frame에서  COM motion을 철저히 분리하는 것을 수반한다. 우리는 모든 입자들이 동등하게 처리되고 모든 입자가 아무런 제약조건 없이 자유롭게 공간을 움직이는 것이 허용되는 internal Cartesian coordinate frame(CCF)을 정의한다. 
laboratory CCF에서 internal CCD로의 좌표 변환은 다음 그림에 묘사되어 있다.


월요일 그림 그리기..


internal Hailtonian은 다음과 같다.
$$ \hat{H}_{int} = -\frac{1}{2} \left( \sum_i^n \frac{1}{\mu_i} \nabla_{r_i}^{2} + \sum_{i\ne j}^n \frac{1}{m_o}\nabla'_{r_i}\nabla_{r_j} \right) +\sum_{i=1}^n \frac{q_0q_i}{r_i}+ \sum_{i>j=1}^n \frac{q_iq_j}{r_ij} $$


#### Explicitly correlated all-particle Gaussian functions

### Energy minimization

#### Rayleigh-Ritz method
Rayleigh-Ritz method는 matrix의 eigenvalue를 빠르게 근사하는 방법으로 다음과 같은 과정을 거친다.

먼저 $N*N$의 matrix A가 $AX = \lambda X$가 성립하는 eigenvalue와 eigenvactor를 가지고, N<m 일 때 $N*m$의 임의의 orthonormal한 column을 가진 matrix V가 있을때 다음과 같은 과정이 성립한다.

1. 먼저 $m*m$의 $V^*AV$ 를 얻는다. 이때  $V^*$는 V의 complex-conjugate이다. 
2. 1에서 구한 matrix의 eigenvector와 eigenvalue를 구한다.
$$
V^{*}AVy_{i} = \mu_i y_i
$$
3. 구한 eigenvector와 eigenvalue로 Ritz vector $\tilde{x_i}$와 Ritz value $\tilde{\lambda_i}$를 구한다. 여기에서 Ritz value $\tilde{\lambda_i}$는 $\mu_i$와 같다.
$$
\tilde{x_i}=Vy_i
$$

4. 결과로 나온 근사 ($\tilde{x_i}$, $\tilde{\lambda_i}$)는 Ritz pair로 불리고, matirx A의 eigenvalue와 eigenvector이다.

예를 들어 다음과 같은 matrix A가 있을때
$$
A = \begin{bmatrix} 2 & 0 & 0 \\ 0 & 2 & 1 \\ 0 & 1 & 2 \end{bmatrix}
$$
그리고 A가 eigenvalue 1, 2, 3이고 각각 다음과 같은 eigenvector를 가지고 있다면
$$
X_{\lambda=1} = \begin{bmatrix} 0 \\ 1 \\ -1 \end{bmatrix},\ 
X_{\lambda=2} = \begin{bmatrix} 1 \\ 0 \\ 0 \end{bmatrix},\  
X_{\lambda=3} = \begin{bmatrix} 0 \\ 1 \\ 1 \end{bmatrix}
$$
이때 다음과 같은 V를 생각하자
$$
V = \begin{bmatrix} 0 & 0 \\ 1 & 0 \\ 0 & 1 \end{bmatrix}
$$
이제 1번과정으로 $V^*AV$를 얻으면 다음과 같다.
$$
V^*AV = \begin{bmatrix} 2 & 1 \\ 1 & 2 \end{bmatrix}
$$
다음으로 2번 과정으로 eigenvector와 eigenvalue를 구하면 eigenvalue는 1, 3이고 각각 다음과 같은 eigenvector를 갖는다.
$$
y_{\mu=1} = \begin{bmatrix} 1 \\ -1 \end{bmatrix},\ 
y_{\mu=3} = \begin{bmatrix} 1 \\ 1 \end{bmatrix}
$$
다음으로 3번 과정 $\tilde{x_i}$의 Ritz vector와 Ritz value를 구한다. 먼저 Ritz value $\tilde{\lambda_i}$는 위의서 구한 eigenvalue 1, 3이고 Ritz vector $\tilde{x_i}$는 다음과 같다.
$$
\tilde{x}_{\tilde{\lambda}=1} = \begin{bmatrix} 0 \\ 1 \\ -1 \end{bmatrix},\ 
\tilde{x}_{\tilde{\lambda}=3} = \begin{bmatrix} 0 \\ 1 \\ 1 \end{bmatrix}
$$
위의 matirx A의 eigenvalue와 eigenvector와 비교하면 Ritz vector와 Ritz value가 일부임을 확인할 수 있다.

## 6.2. Vibrational frequency analysis calculations

### Modeling the vibrational-rotational levels of a diatomic molecule

In [ ]:
# Modeling a diatomic molecule

##### Figure 6.5 - Potential energy(black parabola) and spatial probabilites (color) of the normalized quantum harmonic oscillator wavefunctions for $ v = 0 $ to $ v = 4 $

In [ ]:
# Modeling a diatomic molecule

##### Figure 6.6 - Vibrational energy levels of the hydrogen molecule for $v = 0, \ldots, 0, J = 0$ and $J = 1$

### Computing all vibrational-rotational levels of a molecule

#### Optimizing the geometry of the molecule

#### Calculating a force constant Hessian matrix

#### Converting to mass weighted Cartesian coordinates

#### Diagnalizing the mass weightened Hesssian matrix

In [ ]:
# Code here?

## 6.3. Vibrational spectra for ortho-para isomerization of hydrogen molecules

In [ ]:
# Code here?

## Summary